# Analyze GPT using Embeddings

## Load ChatGPT

The following is an example of an analysis of data collected from GPT-3.5 from Open AI. It was collected using the OpenAI python API below and can be analyzed in Phoenix. This notebook will cover:

* How to import prompt/response pairs collected from GPT
* How to load the dataset into Phoenix for analysis 
* How to collect prompt/response pairs using the openAI python SDK

⚠️ Generating embeddings is very fast on GPU instances (seconds) but can take a 2-3 minute on a CPU. We recommend using a GPU runtime instance for this notebook.


In [ ]:
import pandas as pd

In [ ]:
conversations_df = pd.read_csv(
    "https://storage.googleapis.com/arize-assets/fixtures/Embeddings/GENERATIVE/dataframe_llm_gpt.csv"
)

In [ ]:
import re

import numpy as np


def string_to_array(s):
    numbers = re.findall(r"[-+]?\d*\.\d+|[-+]?\d+", s)
    return np.array([float(num) for num in numbers])

In [ ]:
conversations_df["prompt_vector"] = conversations_df["prompt_vector"].apply(string_to_array)
conversations_df["response_vector"] = conversations_df["response_vector"].apply(string_to_array)

In [ ]:
conversations_df

Installing Arize to make use of the embeddings generators available for use from the SDK generators package

In [ ]:
!pip install arize

In [ ]:
!pip install 'arize[AutoEmbeddings]'

In [ ]:
from arize.pandas.embeddings import EmbeddingGenerator, UseCases

generator = EmbeddingGenerator.from_use_case(
    use_case=UseCases.NLP.SEQUENCE_CLASSIFICATION,
    model_name="distilbert-base-uncased",
    tokenizer_max_length=512,
    batch_size=100,
)

Generate embeddings for each Prompt and Response column

In [ ]:
# Very fast on GPU (seconds) but can take a 2-3 minute on a CPU
conversations_df = conversations_df.reset_index(drop=True)
if not all(col in conversations_df.columns for col in ["prompt_vector", "response_vector"]):
    conversations_df["prompt_vector"] = generator.generate_embeddings(
        text_col=conversations_df["prompt"]
    )
    conversations_df["response_vector"] = generator.generate_embeddings(
        text_col=conversations_df["response"]
    )

**Install Phoenix**

In [ ]:
!pip install "arize-phoenix>=3.20.0"

In [ ]:
import phoenix as px

# Define a Schema() object for Phoenix to pick up data from the correct columns for logging
schema = px.Schema(
    feature_column_names=[
        "step",
        "conversation_id",
        "api_call_duration",
        "response_len",
        "prompt_len",
    ],
    prompt_column_names=px.EmbeddingColumnNames(
        vector_column_name="prompt_vector", raw_data_column_name="prompt"
    ),
    response_column_names=px.EmbeddingColumnNames(
        vector_column_name="response_vector", raw_data_column_name="response"
    ),
)

In [ ]:
# Create the dataset from the conversation dataframe & schema
conv_ds = px.Inferences(conversations_df, schema, "production")

In [ ]:
# Click the link below to open in a view in Phoenix of ChatGPT data
px.launch_app(conv_ds)

**Collecting GPT Prompt & Response Data**

In order to analyze data in Phoenix the dataframe in the pervious section was collected using the code below.

In [ ]:
!pip install "openai>=1"

In [ ]:
import getpass
import os

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
import time
import uuid

from openai import OpenAI

client = OpenAI()

messages = []

In [ ]:
print("This is the ChatGPT interface. Type in prompt!")
print("the prompt/response data will be concatenated with conversations_df dataframe.")
print("This cell can be run many times to concatenate more data.")
print("To Exit type: CTRL-D")


def count_words(text):
    words = text.split()
    return len(words)


try:
    data = {
        "prompt": [],
        "response": [],
        "step": [],
        "conversation_id": [],
        "prediction_id": [],
        "api_call_duration": [],
        "prompt_len": [],
        "response_len": [],
    }
    # This represents a single string of a conversation
    conversation_id = uuid.uuid4().hex[:4]
    step = 0
    while True:
        message = input("Prompt : ")
        start_time = time.time()
        if message:
            messages.append(
                {"role": "user", "content": message},
            )
            chat = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
        end_time = time.time()
        reply = chat.choices[0].message.content
        data["prediction_id"].append(str(uuid.uuid4())[:20])
        data["prompt"].append(message)
        data["response"].append(reply)
        data["step"].append(step)
        data["conversation_id"].append(conversation_id)
        data["api_call_duration"].append(end_time - start_time)
        data["prompt_len"].append(
            count_words(message)
        )  # Words / not tokens, but just a simple example
        data["response_len"].append(
            count_words(reply)
        )  # Words / not tokens, but just a simple example
        print(str(end_time - start_time))
        step += 1
        print(f"ChatGPT Response: {reply}")
        messages.append({"role": "assistant", "content": reply})
except Exception as e:
    print("Exiting Chat")
    print(str(e))
    df = pd.DataFrame(data)
    conversations_df = pd.concat([conversations_df, df])
    messages = []

In [ ]:
conversations_df = conversations_df.reset_index(drop=True)
conversations_df.head()

The example above is just for test purposes and application specific integrations will look different. 